In [47]:
import os
import openai
    
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [3]:
def get_completion(prompt, model=llm_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]

In [4]:
# Templates
customer_message = """ Me podrían indicar por favor cual es el horario \
    de atención de la IPS Cuidarte? Necesito sacar una cita \
    con medico general
"""
style = """ Colombian spanish \
    with respectful calm tone
"""
prompt = """ Translate the text \
    into a style that is {style}, \
    text: {text}
"""
prompt1 = """ From the given text, extract the following information: \
    company = The company of the request \
    type = The kind of appoitment \
    
    Format the output as JSON with following the next keys:
    company
    type
    text: {text}
    """

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [6]:
chat = ChatOpenAI(temperature = 0) # Temperature = 0 means less randomness
prompt_template = ChatPromptTemplate.from_template(prompt)

# Look for template inputs
prompt_template.messages[0].prompt.input_variables

#Assigning input variables
customer_messages = prompt_template.format_messages(style=style, text=customer_message)

In [7]:
print(customer_messages[0])

content=' Translate the text     into a style that is  Colombian spanish     with respectful calm tone\n,     text:  Me podrían indicar por favor cual es el horario     de atención de la IPS Cuidarte? Necesito sacar una cita     con medico general\n\n' additional_kwargs={} example=False


In [8]:
customer_response = chat(customer_messages)
print(customer_response.content)

¿Podrían por favor informarme cuál es el horario de atención de la IPS Cuidarte? Necesito agendar una cita con el médico general.


In [9]:
# Retrive information from a message
prompt_template = ChatPromptTemplate.from_template(prompt1)

#Assigning input variables
customer_messages = prompt_template.format_messages(text=customer_message)
customer_response = chat(customer_messages)
print(customer_response.content)

{
  "company": "IPS Cuidarte",
  "type": "medico general"
}


In [28]:
# Defining an output schema
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

company_schema = ResponseSchema(name = "company", description="The company of the request")
type_schema = ResponseSchema(name = "type", description="The kind of the appointment")

response_schema = [company_schema, type_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schema)

In [34]:
format_instructions = output_parser.get_format_instructions(response_schema)
print(format_instructions)


```json
{
	"company": string  // The company of the request
	"type": string  // The kind of the appointment
}



In [42]:
prompt2 = """\
For the following text, extract the following information:

company: The company of the request \
If this information is not found, output -1.

type: The kind of appoitment\
If this information is not found, output -1,\

and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

In [43]:
prompt_template = ChatPromptTemplate.from_template(prompt2)

#Assigning input variables
messages = prompt_template.format_messages(text=customer_message, 
                                format_instructions=format_instructions)
print(messages[0].content)


For the following text, extract the following information:

company: The company of the request If this information is not found, output -1.

type: The kind of appoitmentIf this information is not found, output -1,
and output them as a comma separated Python list.

text:  Me podrían indicar por favor cual es el horario     de atención de la IPS Cuidarte? Necesito sacar una cita     con medico general



```json
{
	"company": string  // The company of the request
	"type": string  // The kind of the appointment
}




In [44]:
response = chat(messages)
print(response.content)

{
    "company": "IPS Cuidarte",
    "type": "medico general"
}


In [45]:
output_dict = output_parser.parse(response.content)
print(f"dict is {output_dict}")

dict is {'company': 'IPS Cuidarte', 'type': 'medico general'}
